# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096998


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:14,  2.66s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:03,  2.34s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:43,  1.67s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:30,  1.21s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  8.32it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  7.56it/s]

Rendering models:  97%|█████████▋| 29/30 [00:09<00:00,  7.09it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  7.71it/s]

Jnursssmith5263                       0.001877
Kartoon1x                             0.021802
not-logged-in-46a23861c6275e310c67    0.037463
Isterica                              0.000915
jnarayanbvg                           0.001023
not-logged-in-085368c0fc90e59be41c    0.007412
awright5                              0.002138
thiagoxxxx                            0.000478
JoshMcflurry                          0.005978
AaronWallage117                       0.106312
Jessica_Korkmaz                       0.007939
Lover_girl99                          0.005400
jayton                                0.000351
Maverna                               0.002224
AlexYoung35                           0.000188
gabrielweiss                          0.004758
not-logged-in-214814e52004b42491a4    0.001302
RaulE                                 0.009799
Lavadude                              0.012662
not-logged-in-90294de164951e098d0b    0.003421
W7252291                              0.002818
Tessaje04    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())